In [ ]:
from IPython.display import display,HTML
def dhtml(st):
    display(HTML("""<style>
    @import url('https://fonts.googleapis.com/css?family=Roboto|Ewert&effect=3d');      
    </style><p class='font-effect-3d' onclick='setStyle(this,"#00ff66")'
    style='font-family:Roboto; font-size:25px; color:#ff355e;'>
    %s</p>"""%st+"""<script>
    function setStyle(element,c) {
     var docs=document.getElementsByClassName('font-effect-3d');
     for (var i=0; i<docs.length; i++) {
         docs[i].style='font-family:Ewert; font-size:22px;'; 
         docs[i].style.color=c;}; };
    </script>"""))
dhtml('Code Modules & Helpful Functions')

In [ ]:
import numpy as np,pylab as pl,pandas as pd
import h5py,tensorflow as tf
import tensorflow_hub as th

In [ ]:
def premodel(pixels,dense,mh,labels):
    model=tf.keras.Sequential([
        tf.keras.layers.Input((pixels,pixels,3),
                              name='input'),
        th.KerasLayer(mh,trainable=True),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(dense,activation='relu'),
        tf.keras.layers.Dropout(rate=.5),
        tf.keras.layers.Dense(labels,activation='softmax')])
    model.compile(optimizer='adam',metrics=['accuracy'],
                  loss='sparse_categorical_crossentropy')
    display(model.summary())
    return model
fw='weights.best.hdf5'
def cb(fw):
    early_stopping=tf.keras.callbacks\
    .EarlyStopping(monitor='val_loss',patience=20,verbose=2)
    checkpointer=tf.keras.callbacks\
    .ModelCheckpoint(filepath=fw,save_best_only=True,verbose=2)
    lr_reduction=tf.keras.callbacks\
    .ReduceLROnPlateau(monitor='val_loss',verbose=2,
                       patience=5,factor=.8)
    return [checkpointer,early_stopping,lr_reduction]
def display_resize(x_train,x_valid,x_test,
                   y_valid,pixels):
    x_train=tf.image.resize(x_train,[pixels,pixels])
    x_valid=tf.image.resize(x_valid,[pixels,pixels])
    x_test=tf.image.resize(x_test,[pixels,pixels])
    img=x_valid[1]
    lbl='One example of resized images \nlabel: '+\
     str(y_valid[1][0])+'\nshape: '+str(img.shape)
    pl.imshow(img); pl.title(lbl)
    return [x_train,x_valid,x_test]

In [ ]:
dhtml('Data Loading & Preprocessing')

In [ ]:
fpath='../input/classification-of-handwritten-letters/'
f='LetterColorImages_123.h5'
f=h5py.File(fpath+f,'r')
keys=list(f.keys()); print(keys)
x=np.array(f[keys[1]],dtype='float32')/255
y=np.array(f[keys[2]],dtype='int32')\
.reshape(-1,1)-1
N=len(y); n=int(.1*N)
shuffle_ids=np.arange(N)
np.random.RandomState(23).shuffle(shuffle_ids)
x,y=x[shuffle_ids],y[shuffle_ids]
x_test,x_valid,x_train=x[:n],x[n:2*n],x[2*n:]
y_test,y_valid,y_train=y[:n],y[n:2*n],y[2*n:]
x_valid.shape,y_valid.shape

In [ ]:
dhtml('Pre-Trained Saved Models')

In [ ]:
[handle_base,pixels]=["mobilenet_v2_050_96",96]
dhtml('#1 '+handle_base)
mhandle="https://tfhub.dev/google/imagenet/{}/feature_vector/4"\
.format(handle_base)

In [ ]:
[x_train1,x_valid1,x_test1]=\
display_resize(x_train,x_valid,x_test,
               y_valid,pixels)

In [ ]:
model=premodel(pixels,1024,mhandle,33)

In [ ]:
history=model.fit(x=x_train1,y=y_train,batch_size=64,
                  epochs=50,callbacks=cb(fw),
                  validation_data=(x_valid1,y_valid))

In [ ]:
model.load_weights(fw)
model.evaluate(x_test1,y_test)

In [ ]:
[handle_base,pixels]=["mobilenet_v2_075_96",96]
dhtml('#2 '+handle_base)
mhandle="https://tfhub.dev/google/imagenet/{}/feature_vector/4"\
.format(handle_base)

In [ ]:
model=premodel(pixels,1024,mhandle,33)

In [ ]:
history=model.fit(x=x_train1,y=y_train,batch_size=64,
                  epochs=50,callbacks=cb(fw),
                  validation_data=(x_valid1,y_valid))

In [ ]:
model.load_weights(fw)
model.evaluate(x_test1,y_test)

In [ ]:
[handle_base,pixels]=["mobilenet_v1_100_128",128]
dhtml('#3 '+handle_base)
mhandle="https://tfhub.dev/google/imagenet/{}/feature_vector/4"\
.format(handle_base)

In [ ]:
[x_train1,x_valid1,x_test1]=\
display_resize(x_train,x_valid,x_test,
               y_valid,pixels)

In [ ]:
model=premodel(pixels,1024,mhandle,33)

In [ ]:
history=model.fit(x=x_train1,y=y_train,batch_size=64,
                  epochs=50,callbacks=cb(fw),
                  validation_data=(x_valid1,y_valid))

In [ ]:
model.load_weights(fw)
model.evaluate(x_test1,y_test)

In [ ]:
[handle_base,pixels]=["mobilenet_v2_050_128",128]
dhtml('#4 '+handle_base)
mhandle="https://tfhub.dev/google/imagenet/{}/feature_vector/4"\
.format(handle_base)

In [ ]:
model=premodel(pixels,1024,mhandle,33)

In [ ]:
history=model.fit(x=x_train1,y=y_train,batch_size=64,
                  epochs=50,callbacks=cb(fw),
                  validation_data=(x_valid1,y_valid))

In [ ]:
model.load_weights(fw)
model.evaluate(x_test1,y_test)